In [1]:
import sys
sys.path.append('./scripts')

In [2]:
sys.version

'3.5.1 |Continuum Analytics, Inc.| (default, Dec  7 2015, 11:24:55) \n[GCC 4.2.1 (Apple Inc. build 5577)]'

In [3]:
import numpy as np

In [4]:
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor()


In [5]:
np.loadtxt('../data/Ht.txt',skiprows=0, dtype=np.float64)

array([[ -2.81467040e-07,  -2.34740700e-07,  -2.09770750e-07, ...,
         -1.03084960e-07,  -8.31573740e-08,  -7.13508260e-08],
       [ -3.16907890e-07,  -2.75000420e-07,  -2.47578670e-07, ...,
         -1.17969640e-07,  -1.00952460e-07,  -9.04247490e-08],
       [ -3.38413540e-07,  -3.15388320e-07,  -3.05160400e-07, ...,
         -1.54530500e-07,  -1.36889770e-07,  -1.25013000e-07],
       ..., 
       [ -5.69642000e-07,  -5.58196580e-07,  -5.38326900e-07, ...,
         -2.09608030e-07,  -2.10239930e-07,  -1.95144070e-07],
       [ -5.87961440e-07,  -5.74981360e-07,  -5.34893270e-07, ...,
         -1.52148900e-07,  -1.56362660e-07,  -1.43316290e-07],
       [ -6.00122410e-07,  -5.87407560e-07,  -5.40756450e-07, ...,
         -1.16781640e-07,  -1.01914790e-07,  -8.63592080e-08]])

In [6]:
Ht = np.loadtxt('../data/Ht.txt',skiprows=0, dtype=np.float64)
Po = np.loadtxt('../data/Po.txt',skiprows=0, dtype=np.float64)
Ph = np.loadtxt('../data/Ph.txt',skiprows=0, dtype=np.float64)
Am = np.loadtxt('../data/Am.txt',skiprows=0, dtype=np.float64)
Pl = np.loadtxt('../data/Pl.txt',skiprows=0, dtype=np.float64)

In [7]:
# flatten the images
Ht_flat = Ht.flatten()
Po_flat = Po.flatten()
Ph_flat = Ph.flatten()
Am_flat = Am.flatten()
Pl_flat = Pl.flatten()

In [8]:
X = [Ht_flat, Po_flat, Ph_flat, Am_flat]
X = np.array(X).T
Y = np.array(Pl_flat).T
print(X.shape)
print(Y.shape)
Y

(65536, 4)
(65536,)


array([ 3201.,  3166.,  3208., ...,  3810.,  4895.,  7214.])

In [9]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, random_state=0)
clf = DecisionTreeRegressor(max_depth=11)
clf.fit(Xtrain, ytrain)
ypred = clf.predict(Xtest)

In [10]:
metrics.mean_squared_error(ytest, ypred)

133044735.76203227

In [11]:
print(clf)

DecisionTreeRegressor(criterion='mse', max_depth=11, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')


In [12]:
# We can export to a graphviz file, if we want to: 
with open("./results/afm_tree.dot", 'w') as f:
    f = tree.export_graphviz(clf, out_file=f)
    
# Note that the sklearn tutorial uses pydot to turn .dot into .pdf; 
# It isn't supported in Python 3.  
# Instead figure out how to do it with https://pypi.python.org/pypi/graphviz, 
# or just take the .dot straight into d3 as Matt showed. 

## Demo of sklearn tree --> dict --> json

In [13]:
import tree_to_dict

In [14]:
tree_to_dict.rules(clf, ['Ht2_flat', 'Po2_flat', 'Ph2_flat', 'Am2_flat'], 'photoluminescence')

{'children': [{'children': [{'children': [{'children': [{'children': [{'children': [{'children': [{'children': [{'children': [{'children': [{'children': [{'name': '6515 of p'},
                      {'name': '5362 of p'}],
                     'name': 'Po2_flat > 0.08500814437866211'},
                    {'name': '2723 of p'}],
                   'name': 'Ph2_flat > 69.14630126953125'},
                  {'children': [{'children': [{'name': '7654 of p'},
                      {'name': '8467 of p'}],
                     'name': 'Ph2_flat > 72.55138397216797'},
                    {'children': [{'name': '5930 of p'},
                      {'name': '6735 of p'}],
                     'name': 'Ph2_flat > 73.4109878540039'}],
                   'name': 'Po2_flat > 0.08319616317749023'}],
                 'name': 'Po2_flat > 0.08350729942321777'},
                {'children': [{'children': [{'children': [{'name': '9990 of p'},
                      {'name': '3439 of p'}],
                 

In [15]:
# Use Janet's wrapper to save the tree as json:
tree_to_dict.save_tree_as_dict(clf, feature_names=['Ht2_flat', 'Po2_flat', 'Ph2_flat', 'Am2_flat'],
                               label_names='photoluminescence', 
                               save_path='./results/sample_json_from_sklearn.json', node_index=0)